In [1]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
import json
from functools import partial
from itertools import product
from typing import Optional, Literal, Callable
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from collections import defaultdict

import submitit
from submitit.core.utils import CommandFunction
import nevergrad as ng
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from losses.loss_types import LossType
from utils.exp_utils import get_executor, get_executor_local, run_experiments
from utils.utils import conf_to_args

In [3]:
SCRIPT_NAME = "measurement_tampering.py"
EXP_DIR = Path("output/mtd_exps")
EXP_DIR.mkdir(parents=True, exist_ok=True)

In [8]:
seeds = list(range(8))

# TODO: add aux weights based on tuining
method_configs = {
    # TopK
    "TopK 0.1": {"loss_type": LossType.TOPK},
    "DivDis": {"loss_type": LossType.DIVDIS},
    "FT Trusted": {"loss_type": LossType.ERM, "aux_weight": 0.0, "heads": 1}, # Model
    "Probe for Evidence of Tamper": {
        "loss_type": LossType.ERM,  "aux_weight": 0.0, 
        "heads": 1, "source_labels": ["sensors_agree"], "split_source_target": False, 
        "target_only_disagree": True, "freeze_model": True, "lr": 2e-4},
    "Measurement Predictor (baseline)": {
        "loss_type": LossType.ERM, "heads": 1, "train": False, "load_prior_probe": True
    }
}

dataset_configs = {
    "diamonds": {
        "dataset": f"diamonds", 
        "model": f"codegen-350M-mono-measurement_pred-diamonds", 
        "bootstrap_eval": False, 
        "frac_warmup_steps": 0.10, # 64 step, 
        "epochs": 5
    },
   "generated-stories": {
        "dataset": "generated_stories", 
        "model": "pythia-1_4b-deduped-measurement_pred-generated_stories", 
        "bootstrap_eval": True, 
        "micro_batch_size": 2, 
        "max_length": 1536, 
        "feature_dim": 2048, 
        "frac_warmup_steps": 0.15, # 8 steps, 
        "epochs": 4
   }
}
# TODO: update
dataset_method_hparam_map = {
    "diamonds": {
        "TopK 0.1": {"lr": 2e-5, "aux_weight": 1.0},
        "DivDis": {"lr": 2e-5, "aux_weight": 1.0},
        "FT Trusted": {"lr": 2e-5},
        "Probe for Evidence of Tamper": {"lr": 2e-4},
    },
    "generated-stories": {
        "TopK 0.1": {"lr": 1e-6, "aux_weight": 1.0},
        "DivDis": {"lr": 1e-6, "aux_weight": 1.0},
        "FT Trusted": {"lr": 5e-3},
        "Probe for Evidence of Tamper": {"lr": 5e-4},
    }
}

configs = {}
for (ds_name, ds_config), (method_name, method_config), seed in product(dataset_configs.items(), method_configs.items(), seeds):
    configs[(ds_name, method_name, seed)] = {
        **ds_config, 
        **method_config, 
        "seed": seed, 
        **dataset_method_hparam_map[ds_name].get(method_name, {})
    } 
    if ds_name == "diamonds":
        configs[(ds_name, method_name, seed)]["dataset"] += f"-seed{seed}"
        configs[(ds_name, method_name, seed)]["model"] += f"-seed{seed}"

def get_conf_dir(conf_name: tuple):
    ds, method, seed = conf_name
    return f"{ds}_{method}/{seed}"

for conf_name, conf in configs.items():
    exp_dir = get_conf_dir(conf_name)
    conf["exp_dir"] = exp_dir

# Run Experiments

In [ ]:
executor = get_executor(EXP_DIR, mem_gb=32)
jobs = run_experiments(executor, list(configs.values()), SCRIPT_NAME)